In [1]:
import numpy as np

import matplotlib.pyplot as plt
from matplotlib.ticker import (MultipleLocator, AutoMinorLocator)

import pandas as pd 

# importing functions
import sys
sys.path.append('/Users/audreyburggraf/Desktop/THESIS/Functions')

from functions import *
from unit_conversion_functions import *
from fitting_functions import *
from printing_functions import *

In [2]:
# names 
# inj_params_0P: array holding the injected parameters for the 0 planet fit 
# inj_params_1P: array holding the injected parameters for the 1 planet fit 

In [3]:
df = pd.read_csv('/Users/audreyburggraf/Desktop/THESIS/Gaia Data Files/gaia_data.csv')

#  add the absolute gaia K band magnitude and stellar mass
df["M_ks"] = df.ks_m + 5 + 5*np.log10(df.parallax/1000)

# cut off certain range 
df = df[(4 < df.M_ks) & (df.M_ks < 11)].reset_index(drop=True)

# set the stellar mass 
df["stellar_mass"] =  find_star_mass(df.M_ks)

In [4]:
N1 = 70
N2 = 70000

seed = 8

# call function
parameter_result, synthetic_result, model_result, error_result = find_signal_components(df, N1, N2, print_params='both')

# break down the result statement 

# parameter_result includes the no planet and 1 planet parameters in their proper units 
(inj_params_0P, inj_params_1P) = parameter_result

# model_results includes the signal components and times for the synthetic in [uas] and [years]
(prop_ra_synthetic, prop_dec_synthetic, 
 parallax_ra_synthetic, parallax_dec_synthetic, 
 planetary_ra_synthetic, planetary_dec_synthetic, 
 times_synthetic) = synthetic_result

# model_results includes the signal components and times for the model in [uas] and [years]
(prop_ra_model, prop_dec_model, 
 parallax_ra_model, parallax_dec_model, 
 planetary_ra_model, planetary_dec_model, 
 times_model) = model_result

# error_result includes noise in ra and dec direction plus error (same for ra and dec) in [uas]
(noise_ra, noise_dec, errors) = error_result

N for synthetic data:  70
N for model data    :  70000
 
Planetary parameters:
e       : 0.051808070535058814  [unitless]
omega   : 0.5926930861883118  [radians]
Omega   : 0.6228846014825858  [radians]
cos_i   : 0.09590947862585042  [unitless]
m_planet: 9.877355977413515  [Jupiter masses]
P_orb   : 0.7163163880302189  [years]
t_peri  : 0.35680097024947205  [years]
 
Gaia parameters:
alpha0  : 1  [degrees]
delta0  : 2  [degrees]
mu_alpha: 13.538769285974364  [mas/year]
mu_delta: -182.83832022732648  [mas/year]
parallax: 50  [mas]
m_star  : 0.6569865189014348  [M_sun]
x =  247963
 
astrometric_signature( [9.87735598] [0.65698652] [0.71631639] [20.] ) =  [499.41311552] [uas]


In [5]:
signal_ra_obs  = prop_ra_synthetic + parallax_ra_synthetic + planetary_ra_synthetic + noise_ra
signal_dec_obs = prop_dec_synthetic + parallax_dec_synthetic + planetary_dec_synthetic + noise_dec

In [6]:
fitted_params_0P, np_chi_sq = no_planet_fit(inj_params_0P, signal_ra_obs, signal_dec_obs, noise_ra, noise_dec, times_synthetic)

print_parameter_differences(inj_params_0P, fitted_params_0P, return_type='np')

alpha0  : real=1.00000, fitted=0.99442, difference=0.00558
delta0  : real=2.00000, fitted=2.00915, difference=0.00915
mu_alpha: real=13.53877, fitted=13.53765, difference=0.00112
mu_delta: real=-182.83832, fitted=-182.83209, difference=0.00623
parallax: real=50.00000, fitted=50.01203, difference=0.01203


In [8]:
fitted_params_1P, wp_chi_sq = one_planet_fit(inj_params_1P, signal_ra_obs, signal_dec_obs, noise_ra, noise_dec, times_synthetic)

print_parameter_differences(inj_params_1P, fitted_params_1P, return_type='wp')

/Users/audreyburggraf/Desktop/THESIS/Functions/functions.py:457: RuntimeWarning: invalid value encountered in double_scalars
  a = (P**2 * m_star)**(1/3) # [AU]
/Users/audreyburggraf/Desktop/THESIS/Functions/functions.py:144: RuntimeWarning: invalid value encountered in sqrt
  sin_i = np.sqrt(1 - cos_i**2)  # [unitless]
/Users/audreyburggraf/Desktop/THESIS/Functions/functions.py:189: RuntimeWarning: invalid value encountered in sqrt
  Y = np.sqrt((1-e**2))*sin(E) # [unitless]


alpha0  : real=1.00000, fitted=0.99797, difference=0.00203
delta0  : real=2.00000, fitted=2.01308, difference=0.01308
mu_alpha: real=13.53877, fitted=13.53585, difference=0.00292
mu_delta: real=-182.83832, fitted=-182.83857, difference=0.00025
parallax: real=50.00000, fitted=50.01054, difference=0.01054
e       : real=0.05181, fitted=0.99850, difference=0.94669
omega   : real=0.59269, fitted=9.43292, difference=8.84023
Omega   : real=0.62288, fitted=1.12628, difference=0.50339
cos_i   : real=0.09591, fitted=-44.93601, difference=45.03192
m_planet: real=9.87736, fitted=0.50212, difference=9.37523
P_orb   : real=0.71632, fitted=2.79570, difference=2.07938
t_peri  : real=0.35680, fitted=0.40592, difference=0.04912


In [9]:
detection_result = detection_function(np_chi_sq, wp_chi_sq, inj_params_1P, fitted_params_1P, N1)
detection_result

(1147.7975566270302, 410.2410951933886, 737.5564614336415, array([0]))